In [ ]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from matplotlib.pyplot import figure

In [ ]:
save = False
if save:
	matplotlib.use("pgf")
	matplotlib.rcParams.update({
		"pgf.texsystem": "pdflatex",
		'font.family': 'serif',
		'text.usetex': True,
		'pgf.rcfonts': False,
	})

saveLoc = 'graphs/'

In [ ]:
loc = "measuring/"

small = pd.read_csv(loc+"small.csv")
bign = pd.read_csv(loc+'bign.csv')
verybign = pd.read_csv(loc+'verybign.csv')
bigvc = pd.read_csv(loc+'bigvc.csv')

def addLogtime(df):
    df['logtime'] = df['time'].apply(np.log2)
    return df

small = addLogtime(small)
bign = addLogtime(bign)
verybign = addLogtime(verybign)
bigvc = addLogtime(bigvc)

In [ ]:
for er in [0.2, 0.6, 0.8, 1] :
	for n in [10, 30, 60, 100] :
		for q in [2, 5, 9] :
			df = small[ (small['edge_rate'] == er) & (small['n'] == n) & (small['vc'] == q)]
			# display(df)

df = small[ (small['edge_rate'].isin([0.2, 0.6, 0.8, 1]))
	   & (small['n'].isin( [10, 30, 60, 100]))
	   & (small['vc'].isin(  [2, 5, 9] ))
	]
# display(df)

In [ ]:
tmp = small[(small['edge_rate'] == 0.6 )
		&  small['n'].isin(list(range(10, 101, 20)) + [100])
		& (small['vc'] > 1)
	].drop(['logtime'], axis=1).reset_index(drop=True)
tmp['time[ms]'] = tmp['time'].round(1);
tmp = tmp.drop(['time', 'edge_rate'], axis=1)

dfs = np.array_split(tmp, 2)
for df in dfs:
	print(df.to_latex())


In [ ]:
small[small['time'] == small['time'].max()]

In [ ]:
df = small[small['time'].isin(sorted(small['time'], reverse=True)[:5])]
df = df.drop('logtime', axis=1).reset_index(drop=True).rename({'time' : 'time[ms]'}, axis=1)

print(df.to_latex())

In [ ]:
display(bigvc.count())
print(bigvc['time'].max()/1000/60, 'minutes longest instance')
display(bigvc[bigvc['time'] == bigvc['time'].max()])
# display(bigvc['time'].replace(np.nan, 20*60*1000))
print(bigvc['time'].replace(np.nan, 20*60*1000).sum()/1000/60/60, 'hours total')

In [ ]:
df = bigvc[~bigvc.isnull().any(axis=1)]
display(df)


df = df[df['time'].isin(sorted(df['time'], reverse=True)[:5])]
df = df.drop('logtime', axis=1).reset_index(drop=True).rename({'time' : 'time[ms]'}, axis=1)

print(df.to_latex())

In [ ]:
curDF = small[small['vc'] == 9].reset_index(drop=True)
l = []

figure(figsize=(8, 6), dpi=80)
for i in range(1, 11, 2):
	e = i/10
	l.append(f'd = {e}')
	plt.plot(curDF[curDF['edge_rate'] == e]['n'], curDF[curDF['edge_rate'] == e]['time'])
# plt.title('relation with growing edge_rate')
plt.legend(l)
plt.xlabel('n')
plt.ylabel('time[ms]')

if save:
	plt.savefig(saveLoc + 'smallGFixedVC.pgf')
else:
	plt.show()

In [ ]:
# curDF = small[small['edge_rate'] == 0.4].reset_index(drop=True)
# for vc in small['vc'].unique():
# 	plt.plot(curDF[curDF['vc'] == vc]['n'], curDF[curDF['vc'] == vc]['time'])
# 	plt.title('VC = ' + str(vc))
# 	plt.show()

In [ ]:
for e in bign['edge_rate'].unique():
	for vc in bign['vc'].unique():
		df = bign[(bign['edge_rate'] ==e) &  (bign['vc'] == vc)]
		plt.plot(df['n'], df['time'].apply(lambda x : x / 1000))
	plt.legend(['vc = ' + str(vc) for vc in bign['vc'].unique()])
	plt.xlabel('n')
	plt.ylabel('time[s]')
	plt.title(f'edge rate = {e}')
	plt.show()


In [ ]:
for edge_rate in bign['edge_rate'].unique():
	print(edge_rate)
	df = bign[bign['edge_rate'] == edge_rate]
	for vc in df['vc'].unique():
		plt.plot(df[df['vc'] == vc]['n'], df[df['vc'] == vc]['time'])
	plt.legend(df['vc'].unique())
	plt.show()

In [ ]:
for edge_rate in verybign['edge_rate'].unique():
	print(edge_rate)
	df = verybign[verybign['edge_rate'] == edge_rate]
	for vc in df['vc'].unique():
		plt.plot(df[df['vc'] == vc]['n'], df[df['vc'] == vc]['time'], marker='o', linestyle='dashed')
	plt.legend(df['vc'].unique())
	plt.show()

In [ ]:
for e in [0.6]:
	for vc in bigvc['vc'].unique():
		df = bigvc[ (bigvc['edge_rate'] == e) ]
		display(df)
		plt.plot(df['n'], df['time'])
	plt.show()
bigvc

In [ ]:
def texit(df):
	df = df.drop(['logtime'], axis=1).reset_index(drop=True)
	df['time[s]'] = df['time'].apply(lambda x : x /1000).round(1);
	df = df.drop(['time', 'q'], axis=1)
	df = df.fillna('-')
	# df = df.replace("-1", "-")
	print(df.to_latex())

tmp = bigvc[(bigvc['edge_rate'] < 0.6 )]
texit(tmp)
# display(tmp)


tmp = bigvc[(bigvc['edge_rate'] == 0.6 )]
texit(tmp)



# dfs = np.array_split(tmp, 2)
# for df in dfs:
# 	print(df.to_latex())


In [ ]:
df = verybign
df = df[~df.isnull().any(axis=1)]
df = df[df['vc'] == 5]
df['time'] = df['time'].apply(lambda x : x /1000).round(1)
# df['d'] = df['edge_rate']
df = df.rename({'edge_rate' : 'd', 'time' : 'time[s]'}, axis=1)
df['q'] = df['q'].astype(int)
df = df.drop(['logtime'], axis = 1)

print(df.to_latex())